In [1]:
import requests
import pandas as pd
import csv
from numpy.random.mtrand import randint
import time


In [3]:
def zip_codes():
    """method to extract and create a list with all zip codes"""
    filename = '../raw_data/Berlin Zip Codes - Sheet1.csv'
    zip_codes_list = []
    with open(filename, 'r') as csvfile:
        datareader = csv.reader(csvfile)
        for row in datareader:
            zip_codes_list.append(row[3:])
    return zip_codes_list[1:]


# list with all Berlin zipcodes
zip_code_list = zip_codes()

In [76]:
def api_scraper(coords_list):
    """ returns information for all venues in wolt, given a coordinates list"""
    
    headers = {'authority': 'restaurant-api.wolt.com',
        'method': 'GET',
        'path': '/v1/pages/restaurants?lat=52.516913793238444&lon=13.389984460880868',
        'scheme': 'https',
        'accept': 'application/json, text/plain, */*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en,es;q=0.9',
        'app-language': 'en',
        'clientversionnumber': '1.6.151',
        'cookie': 'ravelinDeviceId=rjs-f3df3331-247a-46ac-b626-ec1627b582a2; intercom-id-qwum5ehb=1531a790-2768-4fda-9d8f-41f5f73fe476; intercom-session-qwum5ehb=; _gcl_au=1.1.385640245.1637093317; _fbp=fb.1.1637093318113.856351982; afUserId=07a6dae1-f582-4784-9eb4-924f09ac098b-p; _gac_UA-56809017-2=1.1637093394.Cj0KCQiAys2MBhDOARIsAFf1D1cvfq1acQ1I1tnUILzwSp5QqyaEVycWm41K1fVzZgIvotYve_oDYVQaAoCWEALw_wcB; AF_SYNC=1637700969332; ravelinSessionId=ced613ce-a028-4587-9857-e499b4dbcbf4; _gid=GA1.2.173313073.1638290842; _ga=GA1.2.1935941802.1637093317; _gat_UA-56809017-2=1; _ga_CP7Z2F7NFM=GS1.1.1638300846.26.1.1638300928.38',
        'origin': 'https://wolt.com',
        'platform': 'Web',
        'referer': 'https://wolt.com/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': "macOS",
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36',
        'w-wolt-session-id': '803dea12-029c-4f13-b44c-081b1b423605'}
    
    restaurants_list = []

    base_url = 'https://restaurant-api.wolt.com/v1/pages/restaurants'
    
    for coord in coords_list:
        params = {"lat": coord[0], "lon": coord[1]}
        response = requests.get(base_url, headers=headers, params=params).json()
        try:
            for resto in range(len(response['sections'][0]['items'])):
                resto_dict = dict(
                restaurant_name = response["sections"][0]['items'][resto]['title'],
                # avg_review_score = response['sections'][0]['items'][resto]['venue']['rating']['score'],
                # reviews - TBU
                pricyness = response['sections'][0]['items'][resto]['venue']['price_range'],
                # minimum_order_value - TBU
                delivery_fee = response['sections'][0]['items'][resto]['venue']['delivery_price'][1:],
                # type_of_cuisine = response['sections'][0]['items'][resto]['filtering']['filters'][0]["values"][0],
                # address - TBU
                street = response['sections'][0]['items'][resto]['venue']['address'],
                # zip_code - TBU
                city_name = 'berlin',
                coordinates = response['sections'][0]['items'][resto]['venue']['location'],
                latitude = response['sections'][0]['items'][resto]['venue']['location'][1],
                longitude = response['sections'][0]['items'][resto]['venue']['location'][0],
                link = response['sections'][0]['items'][resto]['link']['target'])

                restaurants_list.append(resto_dict)
                
            print(f" completed {coord}............")

        except Exception:
            print(f" {coord} FAILED!!! ............")

        time.sleep(randint(1, 300)/100)


    restaurants_df = pd.DataFrame(restaurants_list)

    return restaurants_df


In [77]:
wolt_df = api_scraper(zip_code_list)



 completed ['52.517', '13.387']............
 completed ['52.532', '13.385']............
 completed ['52.53', '13.405']............
 completed ['52.521', '13.41']............
 completed ['52.512', '13.416']............
 completed ['52.512', '13.439']............
 ['52.501', '13.465'] FAILED!!! ............
 ['52.516', '13.466'] FAILED!!! ............
 completed ['52.535', '13.426']............
 completed ['52.538', '13.411']............
 completed ['52.545', '13.413']............
 completed ['52.518', '13.32']............
 completed ['52.509', '13.327']............
 completed ['52.497', '13.314']............
 completed ['52.499', '13.326']............
 completed ['52.507', '13.364']............
 completed ['52.508', '13.344']............
 completed ['52.493', '13.397']............
 completed ['52.5', '13.381']............
 completed ['52.503', '13.401']............
 completed ['52.501', '13.436']............
 completed ['52.497', '13.427']............
 completed ['52.518', '13.376']....

In [118]:
wolt_df_clean = wolt_df.drop(columns=['coordinates']).drop_duplicates(subset=[
    'restaurant_name', 'pricyness', 'street', 'city_name',
    'latitude', 'longitude', 'link'], keep="first")
wolt_df_clean = wolt_df_clean.reset_index(drop=True)
wolt_df_clean['coordinates'] = wolt_df_clean.latitude.astype(
    str) + "," + wolt_df_clean.longitude.astype(str)
wolt_df_clean["database"] = "wolt"
wolt_df_clean = wolt_df_clean[[
    'restaurant_name', 'pricyness', 'delivery_fee',
    'street', 'city_name', 'latitude', 'longitude', 'coordinates', 'link',
    'database'
]]
wolt_df_clean.to_csv("../geotracker/data/wolt_restaurants_from_api.csv", index=False)


In [119]:
wolt_df_clean

,restaurant_name,pricyness,delivery_fee,street,city_name,latitude,longitude,coordinates,link,database
0,Burgermeister Kottbusser Tor,1,2.90,Skalitzer Str. 136,berlin,52.499503,13.418597,"52.49950319999998,13.418596600000013",https://wolt.com/en/deu/berlin/restaurant/burg...,wolt
1,Pizza Meister,1,2.90,Zossener Straße 12,berlin,52.492279,13.394353,"52.4922794,13.3943531",https://wolt.com/en/deu/berlin/restaurant/pizz...,wolt
2,Burgermann,1,2.90,Schönhauser Allee 173,berlin,52.532460,13.412156,"52.532460231915074,13.412155674891162",https://wolt.com/en/deu/berlin/restaurant/burg...,wolt
3,Bergmann Burger by BeefMan,1,2.90,Bergmannstraße 112,berlin,52.490227,13.387341,"52.4902266,13.3873411",https://wolt.com/en/deu/berlin/restaurant/berg...,wolt
4,Pizza Noi,1,2.90,Potsdamer Straße 112,berlin,52.500574,13.363153,"52.50057444327202,13.363152750293107",https://wolt.com/en/deu/berlin/restaurant/pizz...,wolt
...,...,...,...,...,...,...,...,...,...,...
1477,MA'LOA Poke Bowl Schloßstraße,2,2.90,Schloßstraße 129,berlin,52.464433,13.328313,"52.46443259999999,13.3283134",https://wolt.com/en/deu/berlin/restaurant/malo...,wolt
1478,Go!Fish - Schlossstrasse,1,2.90,This is a virtual venue,berlin,52.460094,13.323443,"52.460094,13.3234431",https://wolt.com/en/deu/berlin/restaurant/gofi...,wolt
1479,Subway Rheinstrasse,2,1.90,Rheinstraße 66,berlin,52.471697,13.335832,"52.4716974,13.3358317",https://wolt.com/en/deu/berlin/restaurant/subw...,wolt
1480,Risa Chicken - Steglitz,1,2.90,Albrechtstrasse 7,berlin,52.455913,13.323818,"52.4559132,13.3238184",https://wolt.com/en/deu/berlin/restaurant/risa...,wolt
